此处学习 MCMC 法，使用 Gibbs 采样法实现。

# 正态分布

In [3]:
# 给出观测值和其他参数，更新参数mu
update_mu <- function(X, sigma2, prior){
  # X是样本数据，sigma2是上一步迭代中的sigma，prior是超参数的先验值
  n = length(X)
  sumX = sum(X)
  tmp = 1 / (n/sigma2 + 1/prior[2]) # 计算mu的后验分布方差
  mu_n = (sumX/sigma2 + prior[1]/prior[2]) * tmp # 计算mu的后验分布均值
  sigma_n = sqrt(tmp) # mu的后验分布标准差
  # 返回一个从mu的后验分布中的抽样值作为mu的更新值
  return(rnorm(1, mu_n, sigma_n))
}

In [4]:
# 给出观测值和其他参数，更新参数sigma2
update_sigma2 <- function(X, mu, prior){
  # X是样本数据，mu是上一步迭代中的mu，prior是超参数的先验值
  n = length(X)
  shape = 0.5*n +prior[1]  # sigma2的后验分布形状参数
  scale = sum((X-mu)^2)
  scale = scale/2 + prior[2]  # sigma2的后验分布尺度参数
  # 返回一个从sigma2的后验分布中的抽样值即为sigma2的更新值
  return(1/rgamma(1, shape, scale))
}

In [5]:
MCMC_Gaussian <- function(X, n_sample, burn_in, thin, init, prior_mu, prior_sigma){
  # X是输入数据
  # n_sample是采样数量
  # brun_in是燃烧期，MCMC法需要抛弃最开始的样本
  # thin是保存的间隔数，因为原理是马氏链，相邻样本是相关的
  # init是参数初始值
  # prior_mu是mu的先验分布中的超参数值
  # prior_sigma是sigma2 的先验分布中的超参数值

  # 预先生成向量，加快代码速度
  mu_sample = rep(0, n_sample)
  sigma2_sample = rep(0, n_sample)

  # 赋予初值
  mu = init[1]
  sigma2 = init[2]

  # 燃烧期
  # 区分燃烧期和采样期，加快代码运行速度
  for (i in 1:burn_in){
    # 反复更新采样后验均值和方差
    mu = update_mu(X, sigma2, prior_mu)
    sigma2 = update_sigma2(X, mu, prior_sigma)

    # 检查采样间隔，thin不能小于 1
    if (thin < 1) thin = 1

    # 开始采样
    for (i in 1:n_sample){
      for (j in 1:thin){
        mu = update_mu(X, sigma2, prior_mu)
        sigma2 = update_sigma2(X, mu, prior_sigma)
      }
      mu_sample[i] = mu
      sigma2_sample[i] = sigma2
    }

    # 输出列表
    # out[[1]]是mu的模拟值
    # out[[2]]是sigma2 的模拟值
    out = list(mu = mu_sample, sigma2 = sigma2_sample)
    return(out)
  }
}

In [12]:
# 设置种子，便于复现
set.seed(1234)

#模拟从一个均值为1，方差为4的正态分布中抽样
a = rnorm(5000,1,2)  
res = MCMC_Gaussian(
  a,
  n_sample = 100000,
  burn_in = 20000,
  thin = 10,
  init = c(10, 10),
  prior_mu = c(0, 100),
  prior_sigma = c(1, 1)
)
#返回结果res为一个列表

In [19]:
mean(res[[1]])

[1] 0.9873263

In [18]:
mean(res[[2]])

[1] 3.931305